# **Código Base de datos por Manzana (Indicadores integrados, VIV + DENUE)**
Esto será el código para crear la base de datos por manzana. <br>
<br>
No toma ninguna otra BD como base, lo crea desde cero, con los datos censales por manzana.
<br>
<!-- Agrupa, además, los resultados por municipio y por zona metropolitana, generando bases de datos adicionales. -->

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon
from dbfread import DBF
import jenkspy
import json
from jenkspy import jenks_breaks
import time

## Paso dos - Información censal

In [2]:
# #RUTAS ENTRADA - ALEJANDRO (lap)

# #RUTAS CENSOS
# MZNA_2020_salida = r"C:\Users\ADMIN\Desktop\Ejercicio\MZNA_2020_NAC.csv"
# ITER_2020_salida = r"C:\Users\ADMIN\Desktop\Ejercicio\ITER_2020_NAC.csv"
# MZNA_2010_salida = r"C:\Users\ADMIN\Desktop\Ejercicio\MZNA_2010_NAC.csv"
# ITER_2010_salida = r"C:\Users\ADMIN\Desktop\Ejercicio\ITER_2010_NAC.csv"

# # ruta shp base
# SHP_LOC_NAC_ruta = r"C:\Users\Alejandro\Downloads\DATA\SHP\mg_2020_integrado\conjunto_de_datos\00l.shp"
# SHP_MZN_NAC_ruta = r""

# #ruta gpkg fin
# MAPA_TIPO_ruta = r"C:\Users\Alejandro\Downloads\DATA\Tratados\NAC\INT_MZN_NAC\Mzn_Tipologías_NAC.gpkg"


# RUTAS ENTRADA - ALEJANDRO (SSD)

#RUTAS CENSOS
MZNA_2020_ruta = r"Z:\VOCES\DATA\Tratados\NAC\MZNA_2020_NAC.csv"
ITER_2020_ruta = r"Z:\VOCES\DATA\Tratados\NAC\ITER_2020_NAC.csv"
MZNA_2010_ruta = r"Z:\VOCES\DATA\Tratados\NAC\MZNA_2010_NAC.csv"
ITER_2010_ruta = r"Z:\VOCES\DATA\Tratados\NAC\ITER_2010_NAC.csv"

# ruta shp base
SHP_LOC_NAC_ruta = r"Z:\VOCES\DATA\SHP\NAC\00l.shp"
SHP_MZN_NAC_ruta = r"Z:\VOCES\DATA\Tratados\NAC\MZNyLOC_NAC.gpkg"

#ruta gpkg fin
MAPA_TIPO_ruta = r"Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Mzn_Tipologías_NAC.gpkg"

# Ruta del archivo shp de polígonos de contención urbana
polygons_path = r"Z:\VOCES\DATA\PCU_2018_SHP\PCUS_2018.shp"

#CLAVES METRO
CVE_METRO_ruta = r"Z:\VOCES\DATA\Claves_Metro.csv"

#Claves denue
DENUE_ruta = r"Z:\VOCES\DATA\Tratados\NAC\DENUE.gpkg"

In [ ]:
##########CREAR GPKG BASE

# Leer el archivo de polígonos
polygons_gdf = gpd.read_file(polygons_path)[['geometry', 'CALIF_CLAS']]

# Define el CRS proyectado que deseas utilizar
CRS_PROYECTADO = 'EPSG:6372' 

# Leer shp base
SHP_MZN_gdf= gpd.read_file(SHP_MZN_NAC_ruta, layer='MZN')
SHP_LOC_gdf = gpd.read_file(SHP_LOC_NAC_ruta)

# Proyecta el GeoDataFrame a un CRS proyectado
SHP_MZN_gdf = SHP_MZN_gdf.to_crs(CRS_PROYECTADO)
SHP_LOC_gdf = SHP_LOC_gdf.to_crs(CRS_PROYECTADO)
polygons_gdf = polygons_gdf.to_crs(CRS_PROYECTADO)

# Selección de ITER rural
SHP_LOC_gdf_RURAL = SHP_LOC_gdf[SHP_LOC_gdf['AMBITO'] == 'Rural']

# Filtrar entidades rurales que intersectan con los polígonos
SHP_LOC_gdf_RURAL_INTERSECT = gpd.sjoin(SHP_LOC_gdf_RURAL, polygons_gdf, how='inner', predicate='intersects')

# Concatenar ITER rural a la capa existente
gdf_concatenated = gpd.GeoDataFrame(pd.concat([SHP_MZN_gdf, SHP_LOC_gdf_RURAL_INTERSECT], ignore_index=True))

# Guardar la capa concatenada en el GeoPackage base
gdf_concatenated.to_file(MAPA_TIPO_ruta, layer='Mapa_Tipologias', driver='GPKG')

# Leer gpkg base
MAPA_TIPO_gdf = gpd.read_file(MAPA_TIPO_ruta, layer='Mapa_Tipologias')

# LEER INFORMACIÓN CENSAL
MZNA_2020_df = pd.read_csv(MZNA_2020_ruta, dtype=str)
ITER_2020_df = pd.read_csv(ITER_2020_ruta, dtype=str)
MZNA_2010_df = pd.read_csv(MZNA_2010_ruta, dtype=str)
ITER_2010_df = pd.read_csv(ITER_2010_ruta, dtype=str)

# Filtrar censos
MZNA_2020_df_F = MZNA_2020_df[['CVEGEO', 'POBTOT', 'VIVTOT', 'TVIVHAB', 'TVIVPAR', 'TVIVPARHAB', 'VIVPAR_HAB', 'VIVPAR_UT', 'OCUPVIVPAR', 'VIVPAR_DES', 'VPH_1CUART', 'VPH_2CUART', 'VPH_3YMASC']]
ITER_2020_df_F = ITER_2020_df[['CVEGEO', 'POBTOT', 'VIVTOT', 'TVIVHAB', 'TVIVPAR', 'TVIVPARHAB', 'VIVPAR_HAB', 'VIVPAR_UT', 'OCUPVIVPAR', 'VIVPAR_DES', 'VPH_1CUART', 'VPH_2CUART', 'VPH_3YMASC']]
MZNA_2010_df_F = MZNA_2010_df[['CVEGEO', 'POBTOT', 'VIVTOT', 'TVIVHAB', 'TVIVPAR', 'TVIVPARHAB', 'VIVPAR_HAB', 'VIVPAR_UT', 'OCUPVIVPAR', 'VIVPAR_DES', 'VPH_1CUART', 'VPH_2CUART', 'VPH_3YMASC']]
ITER_2010_df_F = ITER_2010_df[['CVEGEO', 'POBTOT', 'VIVTOT', 'TVIVHAB', 'TVIVPAR', 'TVIVPARHAB', 'VIVPAR_HAB', 'VIVPAR_UT', 'OCUPVIVPAR', 'VIVPAR_DES', 'VPH_1CUART', 'VPH_2CUART', 'VPH_3YMASC']]

# Renombrar columnas de ITER_2020_df antes del merge para evitar conflictos

MZNA_2020_df_renamed = MZNA_2020_df_F.rename(columns={
    'POBTOT': 'POPTOT_MZNA_2020',
    'VIVTOT': 'VIVTOT_MZNA_2020',
    'TVIVHAB': 'TVIVHAB_MZNA_2020',
    'TVIVPAR': 'TVIVPAR_MZNA_2020',
    'TVIVPARHAB': 'TVIVPARHAB_MZNA_2020',
    'VIVPAR_HAB': 'VIVPAR_HAB_MZNA_2020',
    'VIVPAR_UT': 'VIVPAR_UT_MZNA_2020',
    'OCUPVIVPAR': 'OCUPVIVPAR_MZNA_2020',
    'VIVPAR_DES': 'VIVPAR_DES_MZNA_2020',
    'VPH_1CUART': 'VIV1C_MZNA_2020',
    'VPH_2CUART': 'VIV2C_MZNA_2020',
    'VPH_3YMASC': 'VIV3C_MZNA_2020'
})

ITER_2020_df_renamed = ITER_2020_df_F.rename(columns={
    'POBTOT': 'POPTOT_ITER_2020',
    'VIVTOT': 'VIVTOT_ITER_2020',
    'TVIVHAB': 'TVIVHAB_ITER_2020',
    'TVIVPAR': 'TVIVPAR_ITER_2020',
    'TVIVPARHAB': 'TVIVPARHAB_ITER_2020',
    'VIVPAR_HAB': 'VIVPAR_HAB_ITER_2020',
    'VIVPAR_UT': 'VIVPAR_UT_ITER_2020',
    'OCUPVIVPAR': 'OCUPVIVPAR_ITER_2020',
    'VIVPAR_DES': 'VIVPAR_DES_ITER_2020',
    'VPH_1CUART': 'VIV1C_ITER_2020',
    'VPH_2CUART': 'VIV2C_ITER_2020',
    'VPH_3YMASC': 'VIV3C_ITER_2020'
})

MZNA_2010_df_renamed = MZNA_2010_df_F.rename(columns={
    'POBTOT': 'POPTOT_MZNA_2010',
    'VIVTOT': 'VIVTOT_MZNA_2010',
    'TVIVHAB': 'TVIVHAB_MZNA_2010',
    'TVIVPAR': 'TVIVPAR_MZNA_2010',
    'TVIVPARHAB': 'TVIVPARHAB_MZNA_2010',
    'VIVPAR_HAB': 'VIVPAR_HAB_MZNA_2010',
    'VIVPAR_UT': 'VIVPAR_UT_MZNA_2010',
    'OCUPVIVPAR': 'OCUPVIVPAR_MZNA_2010',
    'VIVPAR_DES': 'VIVPAR_DES_MZNA_2010',
    'VPH_1CUART': 'VIV1C_MZNA_2010',
    'VPH_2CUART': 'VIV2C_MZNA_2010',
    'VPH_3YMASC': 'VIV3C_MZNA_2010'
})

ITER_2010_df_renamed = ITER_2010_df_F.rename(columns={
    'POBTOT': 'POPTOT_ITER_2010',
    'VIVTOT': 'VIVTOT_ITER_2010',
    'TVIVHAB': 'TVIVHAB_ITER_2010',
    'TVIVPAR': 'TVIVPAR_ITER_2010',
    'TVIVPARHAB': 'TVIVPARHAB_ITER_2010',
    'VIVPAR_HAB': 'VIVPAR_HAB_ITER_2010',
    'VIVPAR_UT': 'VIVPAR_UT_ITER_2010',
    'OCUPVIVPAR': 'OCUPVIVPAR_ITER_2010',
    'VIVPAR_DES': 'VIVPAR_DES_ITER_2010',
    'VPH_1CUART': 'VIV1C_ITER_2010',
    'VPH_2CUART': 'VIV2C_ITER_2010',
    'VPH_3YMASC': 'VIV3C_ITER_2010'
})

MAPA_TIPO_gdf['CVEGEO'] = MAPA_TIPO_gdf['CVEGEO'].astype(str)
MZNA_2020_df_renamed['CVEGEO'] = MZNA_2020_df_renamed['CVEGEO'].astype(str)
ITER_2020_df_renamed['CVEGEO'] = ITER_2020_df_renamed['CVEGEO'].astype(str)
MZNA_2010_df_renamed['CVEGEO'] = MZNA_2010_df_renamed['CVEGEO'].astype(str)
ITER_2010_df_renamed['CVEGEO'] = ITER_2010_df_renamed['CVEGEO'].astype(str)

# Unir campos de los DataFrames al GeoPackage base por CVEGEO
MAPA_TIPO_gdf1 = MAPA_TIPO_gdf.merge(
    MZNA_2020_df_renamed[[
        'CVEGEO', 'POPTOT_MZNA_2020', 'VIVTOT_MZNA_2020', 'TVIVHAB_MZNA_2020', 'TVIVPAR_MZNA_2020', 'TVIVPARHAB_MZNA_2020', 'VIVPAR_HAB_MZNA_2020', 'VIVPAR_UT_MZNA_2020', 'OCUPVIVPAR_MZNA_2020', 'VIVPAR_DES_MZNA_2020', 'VIV1C_MZNA_2020', 'VIV2C_MZNA_2020', 'VIV3C_MZNA_2020']],
    on='CVEGEO', how='left')

MAPA_TIPO_gdf2 = MAPA_TIPO_gdf1.merge(
    ITER_2020_df_renamed[[
        'CVEGEO', 'POPTOT_ITER_2020', 'VIVTOT_ITER_2020', 'TVIVHAB_ITER_2020', 'TVIVPAR_ITER_2020', 'TVIVPARHAB_ITER_2020', 'VIVPAR_HAB_ITER_2020', 'VIVPAR_UT_ITER_2020', 'OCUPVIVPAR_ITER_2020', 'VIVPAR_DES_ITER_2020', 'VIV1C_ITER_2020', 'VIV2C_ITER_2020', 'VIV3C_ITER_2020']],
    on='CVEGEO', how='left')


MAPA_TIPO_gdf3 = MAPA_TIPO_gdf2.merge(
    MZNA_2010_df_renamed[[
        'CVEGEO', 'POPTOT_MZNA_2010', 'VIVTOT_MZNA_2010', 'TVIVHAB_MZNA_2010', 'TVIVPAR_MZNA_2010', 'TVIVPARHAB_MZNA_2010', 'VIVPAR_HAB_MZNA_2010', 'VIVPAR_UT_MZNA_2010', 'OCUPVIVPAR_MZNA_2010', 'VIVPAR_DES_MZNA_2010', 'VIV1C_MZNA_2010', 'VIV2C_MZNA_2010', 'VIV3C_MZNA_2010']],
    on='CVEGEO', how='left')

MAPA_TIPO_gdf4 = MAPA_TIPO_gdf3.merge(
    ITER_2010_df_renamed[[
        'CVEGEO', 'POPTOT_ITER_2010', 'VIVTOT_ITER_2010', 'TVIVHAB_ITER_2010', 'TVIVPAR_ITER_2010', 'TVIVPARHAB_ITER_2010', 'VIVPAR_HAB_ITER_2010', 'VIVPAR_UT_ITER_2010', 'OCUPVIVPAR_ITER_2010', 'VIVPAR_DES_ITER_2010', 'VIV1C_ITER_2010', 'VIV2C_ITER_2010', 'VIV3C_ITER_2010']],
    on='CVEGEO', how='left')

# Reemplazar * con 1
def replace(dataframe):
    return dataframe.replace('*', '1')

# Limpiar *
MAPA_TIPO_gdf4_A = replace(MAPA_TIPO_gdf4)

# Reemplazar N/D
MAPA_TIPO_gdf4_B = MAPA_TIPO_gdf4_A.replace('N/D', np.nan)

# Actualizar valores sin información de "null" a 0
MAPA_TIPO_gdf4_C = MAPA_TIPO_gdf4_B.fillna(0)

# Crear campos POPTOT2010, POPTOT2020, VIVPAR2010 y VIVPAR2020 y llenado respectivamente
MAPA_TIPO_gdf4_C['POPTOT2010'] = MAPA_TIPO_gdf4_C['POPTOT_MZNA_2010'].astype(int) + MAPA_TIPO_gdf4_C['POPTOT_ITER_2010'].astype(int)
MAPA_TIPO_gdf4_C['POPTOT2020'] = MAPA_TIPO_gdf4_C['POPTOT_MZNA_2020'].astype(int) + MAPA_TIPO_gdf4_C['POPTOT_ITER_2020'].astype(int)
MAPA_TIPO_gdf4_C['VIVTOT2010'] = MAPA_TIPO_gdf4_C['VIVTOT_MZNA_2010'].astype(int) +  MAPA_TIPO_gdf4_C['VIVTOT_ITER_2010'].astype(int)
MAPA_TIPO_gdf4_C['VIVTOT2020'] = MAPA_TIPO_gdf4_C['VIVTOT_MZNA_2020'].astype(int) +  MAPA_TIPO_gdf4_C['VIVTOT_ITER_2020'].astype(int)
MAPA_TIPO_gdf4_C['TVIVHAB2010'] = MAPA_TIPO_gdf4_C['TVIVHAB_MZNA_2010'].astype(int) +  MAPA_TIPO_gdf4_C['TVIVHAB_ITER_2010'].astype(int)
MAPA_TIPO_gdf4_C['TVIVHAB2020'] = MAPA_TIPO_gdf4_C['TVIVHAB_MZNA_2020'].astype(int) +  MAPA_TIPO_gdf4_C['TVIVHAB_ITER_2020'].astype(int)
MAPA_TIPO_gdf4_C['TVIVPAR2010'] = MAPA_TIPO_gdf4_C['TVIVPAR_MZNA_2010'].astype(int) +  MAPA_TIPO_gdf4_C['TVIVPAR_ITER_2010'].astype(int)
MAPA_TIPO_gdf4_C['TVIVPAR2020'] = MAPA_TIPO_gdf4_C['TVIVPAR_MZNA_2020'].astype(int) +  MAPA_TIPO_gdf4_C['TVIVPAR_ITER_2020'].astype(int)
MAPA_TIPO_gdf4_C['TVIVPARHAB2010'] = MAPA_TIPO_gdf4_C['TVIVPARHAB_MZNA_2010'].astype(int) +  MAPA_TIPO_gdf4_C['TVIVPARHAB_ITER_2010'].astype(int)
MAPA_TIPO_gdf4_C['TVIVPARHAB2020'] = MAPA_TIPO_gdf4_C['TVIVPARHAB_MZNA_2020'].astype(int) +  MAPA_TIPO_gdf4_C['TVIVPARHAB_ITER_2020'].astype(int)
MAPA_TIPO_gdf4_C['VIVPAR_HAB2010'] = MAPA_TIPO_gdf4_C['VIVPAR_HAB_MZNA_2010'].astype(int) + MAPA_TIPO_gdf4_C['VIVPAR_HAB_ITER_2010'].astype(int)
MAPA_TIPO_gdf4_C['VIVPAR_HAB2020'] = MAPA_TIPO_gdf4_C['VIVPAR_HAB_MZNA_2020'].astype(int) + MAPA_TIPO_gdf4_C['VIVPAR_HAB_ITER_2020'].astype(int)
MAPA_TIPO_gdf4_C['VIVPAR_UT2010'] = MAPA_TIPO_gdf4_C['VIVPAR_UT_MZNA_2010'].astype(int) + MAPA_TIPO_gdf4_C['VIVPAR_UT_ITER_2010'].astype(int)
MAPA_TIPO_gdf4_C['VIVPAR_UT2020'] = MAPA_TIPO_gdf4_C['VIVPAR_UT_MZNA_2020'].astype(int) + MAPA_TIPO_gdf4_C['VIVPAR_UT_ITER_2020'].astype(int)
MAPA_TIPO_gdf4_C['OCUPVIVPAR2010'] = MAPA_TIPO_gdf4_C['OCUPVIVPAR_MZNA_2010'].astype(int) +  MAPA_TIPO_gdf4_C['OCUPVIVPAR_ITER_2010'].astype(int)
MAPA_TIPO_gdf4_C['OCUPVIVPAR2020'] = MAPA_TIPO_gdf4_C['OCUPVIVPAR_MZNA_2020'].astype(int) +  MAPA_TIPO_gdf4_C['OCUPVIVPAR_ITER_2020'].astype(int)
MAPA_TIPO_gdf4_C['VIVPAR_DES2010'] = MAPA_TIPO_gdf4_C['VIVPAR_DES_MZNA_2010'].astype(int) + MAPA_TIPO_gdf4_C['VIVPAR_DES_ITER_2010'].astype(int)
MAPA_TIPO_gdf4_C['VIVPAR_DES2020'] = MAPA_TIPO_gdf4_C['VIVPAR_DES_MZNA_2020'].astype(int) + MAPA_TIPO_gdf4_C['VIVPAR_DES_ITER_2020'].astype(int)
MAPA_TIPO_gdf4_C['VIV1C2010'] = MAPA_TIPO_gdf4_C['VIV1C_MZNA_2010'].astype(int) + MAPA_TIPO_gdf4_C['VIV1C_ITER_2010'].astype(int)
MAPA_TIPO_gdf4_C['VIV1C2020'] = MAPA_TIPO_gdf4_C['VIV1C_MZNA_2020'].astype(int) + MAPA_TIPO_gdf4_C['VIV1C_ITER_2020'].astype(int)
MAPA_TIPO_gdf4_C['VIV2C2010'] = MAPA_TIPO_gdf4_C['VIV2C_MZNA_2010'].astype(int) + MAPA_TIPO_gdf4_C['VIV2C_ITER_2010'].astype(int)
MAPA_TIPO_gdf4_C['VIV2C2020'] = MAPA_TIPO_gdf4_C['VIV2C_MZNA_2020'].astype(int) + MAPA_TIPO_gdf4_C['VIV2C_ITER_2020'].astype(int)
MAPA_TIPO_gdf4_C['VIV3C2010'] = MAPA_TIPO_gdf4_C['VIV3C_MZNA_2010'].astype(int) + MAPA_TIPO_gdf4_C['VIV3C_ITER_2010'].astype(int)
MAPA_TIPO_gdf4_C['VIV3C2020'] = MAPA_TIPO_gdf4_C['VIV3C_MZNA_2020'].astype(int) + MAPA_TIPO_gdf4_C['VIV3C_ITER_2020'].astype(int)

# Convertir campos a enteros
fields_to_convert = ['POPTOT2010', 'POPTOT2020', 'VIVTOT2010', 'VIVTOT2020', 
    'TVIVHAB2010', 'TVIVHAB2020', 'TVIVPAR2010', 'TVIVPAR2020',
    'TVIVPARHAB2010', 'TVIVPARHAB2020', 'VIVPAR_HAB2010', 'VIVPAR_HAB2020',
    'VIVPAR_UT2010', 'VIVPAR_UT2020', 'OCUPVIVPAR2010', 'OCUPVIVPAR2020',
    'VIVPAR_DES2010', 'VIVPAR_DES2020', 'VIV1C2010', 'VIV1C2020', 
    'VIV2C2010', 'VIV2C2020', 'VIV3C2010', 'VIV3C2020']
for field in fields_to_convert:
    MAPA_TIPO_gdf4_C[field] = MAPA_TIPO_gdf4_C[field].astype(int)

# Reordenar columnas
ordered_columns = [
    'CVEGEO', 'POPTOT2010', 'POPTOT2020', 'VIVTOT2010', 'VIVTOT2020', 
    'TVIVHAB2010', 'TVIVHAB2020', 'TVIVPAR2010', 'TVIVPAR2020',
    'TVIVPARHAB2010', 'TVIVPARHAB2020', 'VIVPAR_HAB2010', 'VIVPAR_HAB2020',
    'VIVPAR_UT2010', 'VIVPAR_UT2020', 'OCUPVIVPAR2010', 'OCUPVIVPAR2020',
    'VIVPAR_DES2010', 'VIVPAR_DES2020', 'VIV1C2010', 'VIV1C2020', 
    'VIV2C2010', 'VIV2C2020', 'VIV3C2010', 'VIV3C2020'
] + [col for col in MAPA_TIPO_gdf4.columns if col not in [
    'CVEGEO', 'POPTOT2010', 'POPTOT2020', 'VIVTOT2010', 'VIVTOT2020', 
    'TVIVHAB2010', 'TVIVHAB2020', 'TVIVPAR2010', 'TVIVPAR2020',
    'TVIVPARHAB2010', 'TVIVPARHAB2020', 'VIVPAR_HAB2010', 'VIVPAR_HAB2020',
    'VIVPAR_UT2010', 'VIVPAR_UT2020', 'OCUPVIVPAR2010', 'OCUPVIVPAR2020',
    'VIVPAR_DES2010', 'VIVPAR_DES2020', 'VIV1C2010', 'VIV1C2020', 
    'VIV2C2010', 'VIV2C2020', 'VIV3C2010', 'VIV3C2020'
]]


MAPA_TIPO_gdf4_C = MAPA_TIPO_gdf4_C[ordered_columns]

MAPA_TIPO_gdf4_C = MAPA_TIPO_gdf4_C.drop(columns=[ 'POPTOT_MZNA_2020', 'VIVTOT_MZNA_2020', 'TVIVHAB_MZNA_2020', 'TVIVPAR_MZNA_2020', 
                                              'TVIVPARHAB_MZNA_2020', 'VIVPAR_HAB_MZNA_2020', 'VIVPAR_UT_MZNA_2020',
                                              'OCUPVIVPAR_MZNA_2020', 'VIVPAR_DES_MZNA_2020', 'VIV1C_MZNA_2020', 'VIV2C_MZNA_2020',
                                              'VIV3C_MZNA_2020', 'POPTOT_ITER_2020', 'VIVTOT_ITER_2020', 'TVIVHAB_ITER_2020',
                                              'TVIVPAR_ITER_2020', 'TVIVPARHAB_ITER_2020', 'VIVPAR_HAB_ITER_2020', 'VIVPAR_UT_ITER_2020',
                                              'OCUPVIVPAR_ITER_2020', 'VIVPAR_DES_ITER_2020', 'VIV1C_ITER_2020', 'VIV2C_ITER_2020',
                                              'VIV3C_ITER_2020', 'POPTOT_MZNA_2010', 'VIVTOT_MZNA_2010', 'TVIVHAB_MZNA_2010', 
                                              'TVIVPAR_MZNA_2010', 'TVIVPARHAB_MZNA_2010', 'VIVPAR_HAB_MZNA_2010', 'VIVPAR_UT_MZNA_2010', 
                                              'OCUPVIVPAR_MZNA_2010', 'VIVPAR_DES_MZNA_2010', 'VIV1C_MZNA_2010', 'VIV2C_MZNA_2010',
                                              'VIV3C_MZNA_2010', 'POPTOT_ITER_2010', 'VIVTOT_ITER_2010', 'TVIVHAB_ITER_2010',
                                              'TVIVPAR_ITER_2010', 'TVIVPARHAB_ITER_2010', 'VIVPAR_HAB_ITER_2010', 'VIVPAR_UT_ITER_2010',
                                              'OCUPVIVPAR_ITER_2010', 'VIVPAR_DES_ITER_2010', 'VIV1C_ITER_2010', 'VIV2C_ITER_2010', 'VIV3C_ITER_2010'])

# Guardar el GeoPackage base actualizado
MAPA_TIPO_gdf4_C.to_file(MAPA_TIPO_ruta, layer='BASE_DATOS_NAC_MZN', driver="GPKG")

In [ ]:
#AGRUPAR Y ASIGNAR FILTRO Y PROM POR ZM

#Subir archivo con CVE_METRO
CVE_METRO_df = pd.read_csv(CVE_METRO_ruta, dtype={'CVE_MET': str, 'CVEMUN':str})

# Leer la capa existente del GeoPackage base
MAPA_TIPO_gdf = gpd.read_file(MAPA_TIPO_ruta, layer='BASE_DATOS_NAC_MZN')

#Crear campo CVEMUN
def homologate_municipio_id(row):
    return f"{row['CVE_ENT']}{row['CVE_MUN']}"

# MAPA_TIPO_gdf['CVEMUN'] = MAPA_TIPO_gdf.apply(homologate_municipio_id, axis=1)
MAPA_TIPO_gdf['CVEMUN'] = MAPA_TIPO_gdf.apply(homologate_municipio_id, axis=1)

#Añadir claves a GPKG
MAPA_TIPO_gdf = MAPA_TIPO_gdf.merge(CVE_METRO_df[['CVEMUN', 'CVE_MET', 'NOM_MET', 'TIPO_MET','TIPO_MUN']], on='CVEMUN', how='left')

#FILTRAR MUNICIPIOS ZONAS METROPOLITANAS
valores_filtro = CVE_METRO_df['CVEMUN'].unique()

MAPA_TIPO_gdf = MAPA_TIPO_gdf[(MAPA_TIPO_gdf['CVEMUN'].isin(valores_filtro))]

# Guardar el GeoPackage base actualizado
MAPA_TIPO_gdf.to_file(MAPA_TIPO_ruta, layer= 'BASE_DATOS_METRO_MZN', driver="GPKG")

In [ ]:
# Leer la capa existente del GeoPackage base
MAPA_TIPO_gdf = gpd.read_file(MAPA_TIPO_ruta, layer='BASE_DATOS_METRO_MZN')

# Creación de campos y llenado de información

MAPA_TIPO_gdf['area_ha'] = MAPA_TIPO_gdf.area / 10000
MAPA_TIPO_gdf['FILTRADO'] = MAPA_TIPO_gdf['VIVTOT2020'] / MAPA_TIPO_gdf['area_ha']**2
MAPA_TIPO_gdf['DESHABITACION'] = MAPA_TIPO_gdf['VIVPAR_DES2020'] / MAPA_TIPO_gdf['VIVTOT2020']
MAPA_TIPO_gdf['DIFPOP'] = MAPA_TIPO_gdf['POPTOT2020'] - MAPA_TIPO_gdf['POPTOT2010']
MAPA_TIPO_gdf['DIFVIV'] = MAPA_TIPO_gdf['VIVTOT2020'] - MAPA_TIPO_gdf['VIVTOT2010']
MAPA_TIPO_gdf['RELPOP'] = MAPA_TIPO_gdf['DIFPOP'] / MAPA_TIPO_gdf['POPTOT2010']
MAPA_TIPO_gdf['RELVIV'] = MAPA_TIPO_gdf['DIFVIV'] / MAPA_TIPO_gdf['VIVTOT2010']
MAPA_TIPO_gdf['CVEPOP'] = np.where(MAPA_TIPO_gdf['RELPOP'] < -0.000000001, 'A', 'B')
MAPA_TIPO_gdf['CVEVIV'] = np.where(MAPA_TIPO_gdf['RELVIV'] < -.0703, '1',
                                   np.where((MAPA_TIPO_gdf['RELVIV'] >= -.0703) & (MAPA_TIPO_gdf['RELVIV'] <= .0703), 
                                            '2', '3'))
MAPA_TIPO_gdf['CVE'] = MAPA_TIPO_gdf['CVEPOP'] + MAPA_TIPO_gdf['CVEVIV'].astype(str)
conditions = [
    (MAPA_TIPO_gdf['POPTOT2020'] == 0),
    (MAPA_TIPO_gdf['POPTOT2010'] == 0) & (MAPA_TIPO_gdf['VIVTOT2010'] == 0) & (MAPA_TIPO_gdf['CVE'] == 'B3')
]
choices = ['DH', 'B3\'']
MAPA_TIPO_gdf['TIPOLOGIA'] = np.select(conditions, choices, default=MAPA_TIPO_gdf['CVE'])

MAPA_TIPO_gdf['DENS_POP10'] = MAPA_TIPO_gdf['POPTOT2010'] / MAPA_TIPO_gdf['area_ha']
MAPA_TIPO_gdf['DENS_POP20'] = MAPA_TIPO_gdf['POPTOT2020'] / MAPA_TIPO_gdf['area_ha']

MAPA_TIPO_gdf['DENS_VIV10'] = MAPA_TIPO_gdf['VIVTOT2010'] / MAPA_TIPO_gdf['area_ha']
MAPA_TIPO_gdf['DENS_VIV20'] = MAPA_TIPO_gdf['VIVTOT2020'] / MAPA_TIPO_gdf['area_ha']

MAPA_TIPO_gdf['CUARTOS2010'] = ((MAPA_TIPO_gdf['VIV1C2010']*1) + (MAPA_TIPO_gdf['VIV2C2010']*2) + (MAPA_TIPO_gdf['VIV3C2010']*4))
MAPA_TIPO_gdf['CUARTOS2020'] = ((MAPA_TIPO_gdf['VIV1C2020']*1) + (MAPA_TIPO_gdf['VIV2C2020']*2) + (MAPA_TIPO_gdf['VIV3C2020']*4))

MAPA_TIPO_gdf['CPERC2010'] = MAPA_TIPO_gdf['CUARTOS2010'] / MAPA_TIPO_gdf['POPTOT2010']
MAPA_TIPO_gdf['CPERC2020'] = MAPA_TIPO_gdf['CUARTOS2020'] / MAPA_TIPO_gdf['POPTOT2020']

MAPA_TIPO_gdf['HABXVIV2010'] = MAPA_TIPO_gdf['POPTOT2010'] / MAPA_TIPO_gdf['VIVTOT2010']
MAPA_TIPO_gdf['HABXVIV2020'] = MAPA_TIPO_gdf['POPTOT2020'] / MAPA_TIPO_gdf['VIVTOT2020']

def asignar_subtipologia(row):
    # Valores específicos para deshabitación
    rangos_deshabitacion_grupo = [0, .0703, .1416, .2812]  # Modifica estos valores según tus necesidades
    
# Asignar subtipología según los rangos definidos
    if rangos_deshabitacion_grupo[0] <= row['DESHABITACION'] < rangos_deshabitacion_grupo[1]:
        return 'a'
    elif rangos_deshabitacion_grupo[1] <= row['DESHABITACION'] < rangos_deshabitacion_grupo[2]:
        return 'b'
    elif rangos_deshabitacion_grupo[2] <= row['DESHABITACION'] < rangos_deshabitacion_grupo[3]:
        return 'c'
    elif row['DESHABITACION'] >= rangos_deshabitacion_grupo[3]:
        return 'd'
    else:
        return '0'  # Valor predeterminado

# Crear un nuevo campo 'R_DESHAB' y llenarlo con las subtipologías asignadas
MAPA_TIPO_gdf['R_DESHAB'] = MAPA_TIPO_gdf.apply(asignar_subtipologia, axis=1)

# Crear el campo SUBTIPOLOGÍA
MAPA_TIPO_gdf['SUBTIPOLOGÍA'] = MAPA_TIPO_gdf['TIPOLOGIA'].astype(str) + '-' + MAPA_TIPO_gdf['R_DESHAB'].astype(str)

# # Filtrado de información
# MAPA_TIPO_gdf = MAPA_TIPO_gdf[MAPA_TIPO_gdf['FILTRADO'] >= filtro]

# Guardar el GeoPackage con los nuevos campos
MAPA_TIPO_gdf.to_file(MAPA_TIPO_ruta, layer='Mzn_Subtipologias', driver='GPKG')


In [ ]:
## datos DENUE

# El proceso de unir el DENUE a los manzanas y conteo de empleos, se realiza desde QGIS. La preparación de esa base de datos se realiza en otro código.
# Para este paso se necesita el archivo de manzanas ya con la información cargada.

In [ ]:
# UNIR DENUE CON MANZANAS

# Carga de gpkg con información de empleos y unidades económicas
MAPA_TIPO_gdf = gpd.read_file(MAPA_TIPO_ruta, layer='Mzn_Subtipologias')
SHP_DENUE_gdf = gpd.read_file(DENUE_ruta, layer='Mzn_UnEC_Emp_NAC')

# Realizar el merge seleccionando directamente las columnas necesarias de 'manzanas_gdf'
MAPA_TIPO_gdf = MAPA_TIPO_gdf.merge(
    SHP_DENUE_gdf[['CVEGEO', 'UE_10', 'Emp_10', 'UE_20', 'Emp_20']],
    on='CVEGEO',
    how='left'
)

In [ ]:
#Unión temporal por fuera

In [3]:
#ruta gpkg fin
MAPA_TIPO_ruta = r"Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Mzn_Tipologías_NAC_QGIS.gpkg"

start_time = time.time()
MAPA_TIPO_gdf = gpd.read_file(MAPA_TIPO_ruta, layer='Mzn_Subtipologias_DENUEqgis')
end_time = time.time()
print(f"Tiempo de carga de MAPA_TIPO_gdf: {end_time - start_time:.2f} segundos")

Tiempo de carga de MAPA_TIPO_gdf: 4701.23 segundos


In [4]:
#Tamaño de unidades promedio: Empleados entre UE por manzana
MAPA_TIPO_gdf['TAMAÑO'] = MAPA_TIPO_gdf['Emp_20'] / MAPA_TIPO_gdf['UE_20']

# Calcular las sumas de POPTOT2020 y POPTOT2010 por CVE_MET
sum_emptot2020 = MAPA_TIPO_gdf.groupby('CVEMUN')['Emp_20'].transform('sum')
sum_emptot2010 = MAPA_TIPO_gdf.groupby('CVEMUN')['Emp_10'].transform('sum')

# Calcular TASA_POP_MUN
MAPA_TIPO_gdf['TASA_EMP_MUN'] = (((sum_emptot2020 / sum_emptot2010) ** (1/10)) - 1) * 100
print(8)
#resto de campos
MAPA_TIPO_gdf['DIF_EMP'] = MAPA_TIPO_gdf['Emp_20'] - MAPA_TIPO_gdf['Emp_10']
MAPA_TIPO_gdf['DIF_UEC'] = MAPA_TIPO_gdf['UE_20'] - MAPA_TIPO_gdf['UE_10']
MAPA_TIPO_gdf['REL_EMP'] = MAPA_TIPO_gdf['DIF_EMP'] / MAPA_TIPO_gdf['Emp_10']
MAPA_TIPO_gdf['REL_UEC'] = MAPA_TIPO_gdf['DIF_UEC'] / MAPA_TIPO_gdf['UE_10']
MAPA_TIPO_gdf['CVE_EMP'] = np.where(MAPA_TIPO_gdf['REL_EMP'] < -0.000000001, '1', '2')
MAPA_TIPO_gdf['CVE_UEC'] = np.where(MAPA_TIPO_gdf['REL_UEC'] < -.1, 'A',
                                   np.where((MAPA_TIPO_gdf['REL_UEC'] >= -.1) & (MAPA_TIPO_gdf['REL_UEC'] <= .1), 
                                            'B', 'C'))
MAPA_TIPO_gdf['CVE_ue'] = MAPA_TIPO_gdf['CVE_UEC'].astype(str) + MAPA_TIPO_gdf['CVE_EMP']
conditions = [
    (MAPA_TIPO_gdf['Emp_20'] == 0),
    (MAPA_TIPO_gdf['Emp_10'] == 0) & (MAPA_TIPO_gdf['UE_10'] == 0) & (MAPA_TIPO_gdf['CVE_ue'] == 'C2')
]
choices = ['00', 'C2+']
MAPA_TIPO_gdf['TIPOLOGIA_ue'] = np.select(conditions, choices, default=MAPA_TIPO_gdf['CVE_ue'])

MAPA_TIPO_gdf['DENS_EMP20'] = MAPA_TIPO_gdf['Emp_20'] / MAPA_TIPO_gdf['area_ha']
MAPA_TIPO_gdf['DENS_EMP10'] = MAPA_TIPO_gdf['Emp_10'] / MAPA_TIPO_gdf['area_ha']

MAPA_TIPO_gdf['DENS_UEC20'] = MAPA_TIPO_gdf['UE_20'] / MAPA_TIPO_gdf['area_ha']
MAPA_TIPO_gdf['DENS_UEC10'] = MAPA_TIPO_gdf['UE_10'] / MAPA_TIPO_gdf['area_ha']


def asignar_subtipologia(row):
    # Valores específicos para deshabitación
    rangos_deshabitacion_grupo = [0, 1, 30, 100]  # Modifica estos valores según tus necesidades
    
# Asignar subtipología según los rangos definidos
    if rangos_deshabitacion_grupo[0] <= row['TAMAÑO'] < rangos_deshabitacion_grupo[1]:
        return 'SUE'
    elif rangos_deshabitacion_grupo[1] <= row['TAMAÑO'] < rangos_deshabitacion_grupo[2]:
        return 'MICRO'
    elif rangos_deshabitacion_grupo[2] <= row['TAMAÑO'] < rangos_deshabitacion_grupo[3]:
        return 'MEDIANA'
    elif row['TAMAÑO'] >= rangos_deshabitacion_grupo[3]:
        return 'GRANDE'
    else:
        return '0'  # Valor predeterminado
print(11)

# Crear un nuevo campo 'R_DESHAB' y llenarlo con las subtipologías asignadas
MAPA_TIPO_gdf['R_TAM'] = MAPA_TIPO_gdf.apply(asignar_subtipologia, axis=1)

# Crear el campo SUBTIPOLOGÍA
MAPA_TIPO_gdf['SUBTIPOLOGÍA_ue'] = MAPA_TIPO_gdf['TIPOLOGIA_ue'].astype(str) + '-' + MAPA_TIPO_gdf['R_TAM'].astype(str)
print(12)

# Guardar el GeoPackage con los nuevos campos
MAPA_TIPO_gdf.to_file(MAPA_TIPO_ruta, layer='Mzn_SubTipologias_ue', driver='GPKG')

8
11
12


In [5]:
#ruta gpkg fin
MAPA_TIPO_ruta = r"Z:\VOCES\DATA\Tratados\NAC\INT_MZN_NAC\Mzn_Tipologías_NAC.gpkg"

# Guardar el GeoPackage con los nuevos campos
start_time = time.time()
MAPA_TIPO_gdf.to_file(MAPA_TIPO_ruta, layer='Mzn_SubTipologias_ue', driver='GPKG')
end_time = time.time()
print(f"Tiempo: {end_time - start_time:.2f} s")

Tiempo: 3510.94 s
